In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.6f' % x) #为了直观的显示数字，不采用科学计数法
import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
import numpy as np
np.set_printoptions(suppress = True)
import scipy
import os
import math
import time
import random
import joblib
from joblib import Parallel, delayed
import warnings
from tqdm.notebook import tqdm
import gc

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    print('GPUs: %s' % physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as e:
    print('貌似可能没有GPU')
tf.keras.backend.set_floatx('float32')

from scipy.optimize import fmin

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
data_folder = '../../preprocessed_data/final/'
folders = os.listdir(data_folder)
folders
base_types = ['4g', '5g']
kpis = ['PDCCH', 'PDCPDL', 'PDCPUL', 'PDSCH', 'PUSCH', 'RRC']
cities = ['AFE97F546A10368F', 'C48FDFBFC4072E0E', 'EA5EAA705108BDA0', 'F37F452354AC87C9']

['5g_PDCPUL',
 '5g_PDSCH',
 '5g_RRC',
 '4g_PDCPDL',
 '4g_PDCPUL',
 '4g_RRC',
 '.ipynb_checkpoints',
 '4g_PDSCH',
 '5g_PDCPDL',
 '4g_PDCCH',
 '5g_PUSCH',
 '4g_PUSCH',
 '5g_PDCCH']

In [3]:
input_periods = 24 * 21
n_features = 28
input_cols = ['is_holiday_%s' % i for i in range(n_features)] + ['input_%s' % i for i in range(input_periods)]
output_periods = 24 * 7
output_cols = ['output_%s' % i for i in range(output_periods)]

default_lr = 0.0005

def build_compile_model(loss, use_output_relu=False):
    input_ = keras.Input(shape=n_features + input_periods)
    mean_by_row = tf.reshape(tf.reduce_mean(input_[:, n_features:], axis=1), (-1, 1))
    normalized_input = tf.concat([input_[:, :n_features], input_[:, n_features:] / mean_by_row], axis=1)
    hidden_1 = tf.nn.relu(layers.Dense(4096, use_bias=True)(normalized_input))
    hidden_2 = tf.nn.relu(layers.Dense(4096, use_bias=True)(normalized_input) + layers.Dense(4096, use_bias=True)(hidden_1)) / 2.
    output = (layers.Dense(output_periods, use_bias=True)(normalized_input) + layers.Dense(output_periods, use_bias=True)(hidden_1)
              + layers.Dense(output_periods, use_bias=True)(hidden_2)) / 3.
    output *= mean_by_row
    if use_output_relu:
        output = tf.nn.relu(output)
    model = keras.Model(input_, output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=default_lr)
    model.compile(optimizer, loss)
    return model

# def unknown_base_predict_col(predict_col: np.array):
#     def mape(y):
#         ape = np.abs(predict_col - y) / (predict_col + 0.01)
#         ape[ape > 1.] = 1.
#         return np.mean(ape)
#     return fmin(func=mape,x0=0., disp=0)[0]

# def unknown_base_predict(predict):
#     ret = pd.Series(index=['预测未来%s小时' % i for i in range(1, 169)], dtype=np.float32)
#     for col in ['预测未来%s小时' % i for i in range(1, 169)]:
#         ret[col] = unknown_base_predict_col(predict[col].values)
#     return ret

def predict(model, base_type, kpi, nonzero_predict_for_no_history=False, nonzero_predict_for_no_base=False):
    predict_input_df = pd.read_pickle(os.path.join(data_folder, '%s_%s' % (base_type, kpi), 'regression_predict_inputs.pkl'))
    ret_df = pd.DataFrame(model.predict(predict_input_df[input_cols].values.astype(np.float32)), index=predict_input_df.index,
                       columns=['预测未来%s小时' % i for i in range(1, 169)])
#     fill_value = ret_df.mean()
    fill_value = ret_df.median()
    
    # 对历史数据全空或全为0的小区填充空值
    ret_df.fillna(fill_value, inplace=True)
    
    ret = {}
    for city in cities:
        userlabel_list = pd.read_pickle('../../preprocessed_data/Userlabel_lists_final/%s_%s_%s.pkl' % (base_type, kpi, city))
        ret[city] = ret_df.reindex(userlabel_list.UserLabel).reset_index(drop=False)
        ret[city].fillna(fill_value, inplace=True)
    return ret

In [4]:
if not os.path.exists('提交/result_data'):
    os.makedirs('提交/result_data')

In [5]:
%%time
for base_type in base_types:
    for kpi in kpis:
        model = build_compile_model('mse', True)
        model.load_weights('models/%s_%s.h5' % (base_type, kpi))
        result = predict(model, base_type, kpi, False, False)
        for city in cities:
            result[city].to_csv('提交/result_data/%s_%s_%s.csv' % (base_type, kpi, city), index=False)

CPU times: user 1min 45s, sys: 7.41 s, total: 1min 52s
Wall time: 1min 59s


In [6]:
sample_data = pd.read_csv('提交/result_data/4g_PDCPUL_EA5EAA705108BDA0.csv')
# sample_data[np.isnan(sample_data.预测未来1小时)]
sample_data
dict(sample_data.mean())
# dict(unknown_base_predict(sample_data))

,UserLabel,预测未来1小时,预测未来2小时,预测未来3小时,预测未来4小时,预测未来5小时,预测未来6小时,预测未来7小时,预测未来8小时,预测未来9小时,预测未来10小时,预测未来11小时,预测未来12小时,预测未来13小时,预测未来14小时,预测未来15小时,预测未来16小时,预测未来17小时,预测未来18小时,预测未来19小时,预测未来20小时,预测未来21小时,预测未来22小时,预测未来23小时,预测未来24小时,预测未来25小时,预测未来26小时,预测未来27小时,预测未来28小时,预测未来29小时,预测未来30小时,预测未来31小时,预测未来32小时,预测未来33小时,预测未来34小时,预测未来35小时,预测未来36小时,预测未来37小时,预测未来38小时,预测未来39小时,预测未来40小时,预测未来41小时,预测未来42小时,预测未来43小时,预测未来44小时,预测未来45小时,预测未来46小时,预测未来47小时,预测未来48小时,预测未来49小时,预测未来50小时,预测未来51小时,预测未来52小时,预测未来53小时,预测未来54小时,预测未来55小时,预测未来56小时,预测未来57小时,预测未来58小时,预测未来59小时,预测未来60小时,预测未来61小时,预测未来62小时,预测未来63小时,预测未来64小时,预测未来65小时,预测未来66小时,预测未来67小时,预测未来68小时,预测未来69小时,预测未来70小时,预测未来71小时,预测未来72小时,预测未来73小时,预测未来74小时,预测未来75小时,预测未来76小时,预测未来77小时,预测未来78小时,预测未来79小时,预测未来80小时,预测未来81小时,预测未来82小时,预测未来83小时,预测未来84小时,预测未来85小时,预测未来86小时,预测未来87小时,预测未来88小时,预测未来89小时,预测未来90小时,预测未来91小时,预测未来92小时,预测未来93小时,预测未来94小时,预测未来95小时,预测未来96小时,预测未来97小时,预测未来98小时,预测未来99小时,预测未来100小时,预测未来101小时,预测未来102小时,预测未来103小时,预测未来104小时,预测未来105小时,预测未来106小时,预测未来107小时,预测未来108小时,预测未来109小时,预测未来110小时,预测未来111小时,预测未来112小时,预测未来113小时,预测未来114小时,预测未来115小时,预测未来116小时,预测未来117小时,预测未来118小时,预测未来119小时,预测未来120小时,预测未来121小时,预测未来122小时,预测未来123小时,预测未来124小时,预测未来125小时,预测未来126小时,预测未来127小时,预测未来128小时,预测未来129小时,预测未来130小时,预测未来131小时,预测未来132小时,预测未来133小时,预测未来134小时,预测未来135小时,预测未来136小时,预测未来137小时,预测未来138小时,预测未来139小时,预测未来140小时,预测未来141小时,预测未来142小时,预测未来143小时,预测未来144小时,预测未来145小时,预测未来146小时,预测未来147小时,预测未来148小时,预测未来149小时,预测未来150小时,预测未来151小时,预测未来152小时,预测未来153小时,预测未来154小时,预测未来155小时,预测未来156小时,预测未来157小时,预测未来158小时,预测未来159小时,预测未来160小时,预测未来161小时,预测未来162小时,预测未来163小时,预测未来164小时,预测未来165小时,预测未来166小时,预测未来167小时,预测未来168小时
0,0009423fca6ee800cc9b977c40864927-3,217441.640000,132464.390000,92971.510000,74392.190000,82690.734000,117574.070000,183712.190000,353453.440000,414551.530000,466166.800000,498247.880000,474948.060000,459351.060000,402073.440000,443208.440000,472154.300000,492779.530000,508056.000000,484825.000000,470871.970000,505833.750000,501423.120000,436825.160000,293186.220000,203714.140000,117580.020000,90741.250000,77185.540000,80802.210000,113164.290000,180213.620000,338104.660000,408157.720000,478003.780000,523080.160000,486631.750000,474803.380000,427615.440000,471644.060000,506604.200000,536030.800000,533665.060000,522275.940000,503744.940000,549650.800000,518350.100000,439610.880000,305110.300000,218370.020000,123387.350000,92213.850000,82607.050000,81039.945000,111644.050000,171708.690000,300316.620000,366391.300000,454178.200000,519255.700000,494727.700000,478605.500000,435589.840000,453785.120000,490151.000000,502274.440000,489153.220000,477097.220000,460139.780000,510972.780000,494450.280000,433972.560000,290945.720000,210917.340000,134710.840000,98696.450000,82027.500000,79676.945000,110970.610000,165264.250000,289357.800000,323392.440000,419089.940000,493485.660000,470554.660000,425091.530000,413367.970000,438244.530000,467901.030000,480943.060000,467611.340000,434677.340000,442632.970000,481610.700000,476353.500000,392445.160000,273672.530000,192767.100000,122926.760000,83542.766000,76055.195000,81202.234000,112877.080000,171516.270000,290772.880000,349732.600000,424036.440000,481895.220000,462964.800000,431939.780000,408278.300000,441048.280000,489181.560000,486447.880000,478923.880000,458206.400000,445287.560000,493724.900000,478858.100000,400904.500000,270389.500000,172114.160000,120860.820000,88659.664000,78075.340000,78596.850000,115504.100000,173725.880000,300861.940000,377282.250000,449638.000000,494023.120000,468746.060000,448287.280000,401030.120000,438879.560000,475475.880000,492081.530000,493108.160000,468774.250000,455626.600000,486799.060000,457863.500000,406674.440000,264676.970000,209523.400000,114969.820000,84477.140000,67408.280000,73528.480000,106314.140000,173214.120000,322895.340000,403384.970000,457240.880000,510361.400000,469802.060000,436708.340000,397542.780000,437497.400000,463840.470000,493902.120000,478566.340000,461114.700000,453164.600000,471577.720000,458208.880000,385372.660000,265262.900000
1,0012841fe2dc0967a74ab4f59a3974be-2,342265.90000

{'预测未来1小时': 336201.2016183827,
 '预测未来2小时': 188256.6701016747,
 '预测未来3小时': 118696.14525142238,
 '预测未来4小时': 87792.00289546406,
 '预测未来5小时': 70308.26920735346,
 '预测未来6小时': 78075.07025708411,
 '预测未来7小时': 140568.77652052612,
 '预测未来8小时': 277706.3134565552,
 '预测未来9小时': 325654.27536970173,
 '预测未来10小时': 367239.2075998518,
 '预测未来11小时': 397717.47184917564,
 '预测未来12小时': 425711.9686752502,
 '预测未来13小时': 535546.3963963506,
 '预测未来14小时': 376798.49051089294,
 '预测未来15小时': 353221.22570476105,
 '预测未来16小时': 377837.2480803353,
 '预测未来17小时': 409297.48242469435,
 '预测未来18小时': 505754.17224360874,
 '预测未来19小时': 555972.5592020192,
 '预测未来20小时': 595299.8253961654,
 '预测未来21小时': 703939.3940330677,
 '预测未来22小时': 766378.9102596889,
 '预测未来23小时': 725906.9087446276,
 '预测未来24小时': 539514.5121336236,
 '预测未来25小时': 327179.1777288931,
 '预测未来26小时': 185450.8056654835,
 '预测未来27小时': 118482.24854119211,
 '预测未来28小时': 80969.20313399872,
 '预测未来29小时': 69861.67908707677,
 '预测未来30小时': 76354.19445455632,
 '预测未来31小时': 138549.6936871619,
 '预测未来32